In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install -e .

Obtaining file:///tf/notebooks/EPT/1_taxonomist
  Attempting uninstall: taxonomist
    Found existing installation: taxonomist 0.0.3
    Can't uninstall 'taxonomist'. No files were found to uninstall.
  Running setup.py develop for taxonomist
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
df_csv = pd.read_csv('data/processed/EPT-14/EPT-14-DL-final.csv')
print(f"Number of classes: {len(df_csv['Species_DNA'].unique())}")
print(f"Number of specimen: {len(df_csv['Specimen ID'].unique())}")
for label in df_csv['Species_DNA'].unique():
    print(label)

Number of classes: 14
Number of specimen: 743
Goera pilosa
Potamophylax rotundipennis
Baetis vernus
Baetis rhodani
Caenis horaria
Limnephilus lunatus
Halesus radiatus
Chaetopteryx villosa
Anabolia nervosa
Amphinemura standfussi
Ephemera danica
Athripsodes cinereus
Mystacides azureus
Nemoura cinerea


In [4]:
!python scripts/preprocessing/test_dataset_config.py \
    --data_folder "../10_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path 'data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv' \
    --label "Log_weights" \
    --fold 0

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
train: found 94096 fpaths, 94096 labels
Examples:
[PosixPath('/tf/notebooks/EPT/10_images/4_F12/4_F12_1/1_4_F12_1_2023_12_14-09-51-53-989.PNG'), PosixPath('/tf/notebooks/EPT/10_images/4_F12/4_F12_1/2_4_F12_1_2023_12_14-09-51-54-000.PNG'), PosixPath('/tf/notebooks/EPT/10_images/4_F12/4_F12_1/1_4_F12_1_2023_12_14-09-51-54-013.PNG'), PosixPath('/tf/notebooks/EPT/10_images/4_F12/4_F12_1/2_4_F12_1_2023_12_14-09-51-54-025.PNG'), PosixPath('/tf/notebooks/EP

In [5]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin


True

In [6]:
# Train biomass estimation on all 5 folds of previously created splits.
# 00_EPT-14-biomass-lr was used to determine the best learning rate with the efficientnet_b0 model and the ResNet18 model

for fold in range(5):
    !python scripts/02_train.py \
        --data_folder "../10_images/" \
        --dataset_config "conf/user_datasets.py" \
        --dataset_name "biodiscover" \
        --csv_path 'data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv' \
        --label "Log_weights" \
        --fold {fold} \
        --class_map 'none' \
        --imsize 224 \
        --batch_size 256 \
        --aug 'flips-rotate-keep-aspect' \
        --load_to_memory 'False' \
        --model 'efficientnet_b0' \
        --opt 'adamw' \
        --max_epochs 10 \
        --min_epochs 0 \
        --early_stopping 'False' \
        --early_stopping_patience 50 \
        --criterion 'l1' \
        --lr 0.00001 \
        --auto_lr 'True' \
        --log_dir 'V1_EPT-14-biomass-lr' \
        --out_folder 'outputs' \
        --out_prefix 'V1_EPT-14-biomass-lr' \
        --deterministic 'True' \
        --precision '16-mixed'

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Seed set to 42
None
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in ./wandb/run-20241128_091227-241128-0912-f212
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run V1_EPT-14-biomass-lr_efficientnet_b0_f0_241128-0912-f212
wandb: ⭐️ View project at https://wandb.ai/uni-due/V1_EPT-14-biomass-lr
wandb: 🚀 

Epoch 2: 100%|████████████████████| 368/368 [02:05<00:00,  2.93it/s, v_num=f212]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████████████████| 368/368 [02:05<00:00,  2.92it/s, v_num=f212]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  55%|█████████▉        | 50/91 [00:06<00:05,  7.49it/s]


Epoch 4: 100%|████████████████████| 368/368 [02:05<00:00,  2.92it/s, v_num=f212]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|████████████████████| 368/368 [02:05<00:00,  2.93it/s, v_num=f212]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  59%|██████████▋       | 54/91 [00:06<00:04,  7.77it/s]


Epoch 6: 100%|████████████████████| 368/368 [02:05<00:00,  2.93it/s, v_num=f212]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|████████████████████| 368/368 [02:05<00:00,  2.92it/s, v_num=f212]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  64%|███████████▍      | 58/91 [00:07<00:04,  7.46it/s]


Epoch 9: 100%|████████████████████| 368/368 [02:05<00:00,  2.93it/s, v_num=f212]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  96%|█████████████████▏| 87/91 [00:11<00:00,  7.74it/s]


...

Epoch 0: 100%|████████████████████| 366/366 [02:03<00:00,  2.97it/s, v_num=d99e]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|████████████████████| 366/366 [02:05<00:00,  2.92it/s, v_num=d99e]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████████████████| 366/366 [02:05<00:00,  2.92it/s, v_num=d99e]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation:   0%|                                        | 0/92 [00:00<?, ?it/s]


Epoch 4: 100%|████████████████████| 366/366 [02:05<00:00,  2.92it/s, v_num=d99e]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|████████████████████| 366/366 [02:05<00:00,  2.92it/s, v_num=d99e]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   1%|▏                  | 1/92 [00:00<00:03, 26.36it/s]


Epoch 6: 100%|████████████████████| 366/366 [02:05<00:00,  2.92it/s, v_num=d99e]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|████████████████████| 366/366 [02:05<00:00,  2.92it/s, v_num=d99e]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   3%|▌                  | 3/92 [00:00<00:12,  7.14it/s]


Epoch 8:  60%|████████████        | 221/366 [01:16<00:50,  2.90it/s, v_num=d99e]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# Train biomass estimation on all 5 folds of previously created splits.
for fold in range(5):
    !python scripts/02_train.py \
        --data_folder "../10_images/" \
        --dataset_config "conf/user_datasets.py" \
        --dataset_name "biodiscover" \
        --csv_path 'data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv' \
        --label "Log+1_weights" \
        --fold {fold} \
        --class_map 'none' \
        --imsize 224 \
        --batch_size 256 \
        --aug 'flips-rotate-keep-aspect' \
        --load_to_memory 'False' \
        --model 'efficientnet_b0' \
        --opt 'adamw' \
        --max_epochs 10 \
        --min_epochs 0 \
        --early_stopping 'False' \
        --early_stopping_patience 50 \
        --criterion 'l1' \
        --lr 0.00001 \
        --auto_lr 'True' \
        --log_dir 'V1_EPT-14-biomass-lr+1' \
        --out_folder 'outputs' \
        --out_prefix 'V1_EPT-14-biomass-lr+1' \
        --deterministic 'True' \
        --precision '16-mixed'

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
Seed set to 42
None
wandb: Currently logged in as: philipp-rehsen (uni-due). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in ./wandb/run-20241128_111820-241128-1118-0f97
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run V1_EPT-14-biomass-lr+1_efficientnet_b0_f0_241128-1118-0f97
wandb: ⭐️ View project at https://wandb.ai/uni-due/V1_EPT-14-biomass-lr%2B1
wan

Epoch 2: 100%|████████████████████| 368/368 [02:05<00:00,  2.93it/s, v_num=0f97]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|████████████████████| 368/368 [02:06<00:00,  2.92it/s, v_num=0f97]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  55%|█████████▉        | 50/91 [00:06<00:05,  7.60it/s]


Epoch 4: 100%|████████████████████| 368/368 [02:05<00:00,  2.92it/s, v_num=0f97]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 5: 100%|████████████████████| 368/368 [02:06<00:00,  2.92it/s, v_num=0f97]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:  59%|██████████▋       | 54/91 [00:07<00:04,  7.59it/s]


Epoch 6:  18%|███▋                 | 65/368 [00:23<01:48,  2.80it/s, v_num=0f97]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

Epoch 5: 100%|████████████████████| 367/367 [02:05<00:00,  2.92it/s, v_num=131c]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 6: 100%|████████████████████| 367/367 [02:05<00:00,  2.93it/s, v_num=131c]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 7: 100%|████████████████████| 367/367 [02:05<00:00,  2.92it/s, v_num=131c]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|                           | 0/92 [00:00<?, ?it/s]


Epoch 8: 100%|████████████████████| 367/367 [02:05<00:00,  2.93it/s, v_num=131c]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Epoch 9: 100%|████████████████████| 367/367 [02:05<00:00,  2.93it/s, v_num=131c]
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   2%|▍                  | 2/92 [00:00<00:12,  7.25it/s]


Epoch 9: 100%|████████████████████| 367/367 [02:18<00:00,  2.64it/s, v_num=131c]
/usr/local/lib/python3.8/dist-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:156: `.test(ckpt_path="best")` is called with Trainer configured with multiple `ModelCheckpoint` callbacks. It will use the best checkpoint path from first checkpoint callback.
Restoring states from the checkpoint path at /tf/notebooks/EPT/1_taxonomist/outputs/biodiscover/V1_EPT-14-biomass-lr+1_efficientnet_b0/f3/V1_EPT-14-biomass-lr+1_efficientnet_b0_f3_241128-1233-131c_epoch09_val-loss0.18.ckpt
/usr/local/lib/python3.8/dist-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a f

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /tf/notebooks/EPT/1_taxonomist/outputs/biodiscover/V1_EPT-14-biomass-lr+1_efficientnet_b0/f3/V1_EPT-14-biomass-lr+1_efficientnet_b0_f3_241128-1233-131c_epoch09_val-loss0.18.ckpt
Testing DataLoader 0: 100%|███████████████████| 114/114 [00:14<00:00,  7.60it/s]
────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────
     test/loss_epoch        0.2073034793138504
────────────────────────────────────────────────────────────────────────────────
Best model: /tf/notebooks/EPT/1_taxonomist/outputs/biodiscover/V1_EPT-14-biomass-lr+1_efficientnet_b0/f3/V1_EPT-14-biomass-lr+1_efficientnet_b0_f3_241128-1233-131c_epoch09_val-loss0.18.ckpt | score: 0.1802235245704651
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or c

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

